I wanted to see how noisy the raw spikes actually are, by comparing with "identity trick" spikes

In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal.net_builder import NetBuilder

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
np.random.seed(1)

Y = 8
X = 8
N = X * Y
D = 1
SYN_SPACE = 1
FMAX = 1000 # should determine from syn_tau
BIAS = -2
RUN_TIME = 1 # second

SY = Y // 2 // SYN_SPACE
SX = X // 2 // SYN_SPACE

# taps (or dims) don't really matter for this, but whatever
yx_taps = NetBuilder.create_default_yx_taps(SY, SX, D, bad_syn=np.array([[False] * SY] * SX, dtype=bool))
print(yx_taps.shape)

if D == 1:
    plt.imshow(yx_taps[:, :, 0])
else:
    fig, ax = plt.subplots(1, D, figsize=(D*3,3))
    for d in range(D):
        ax[d].imshow(yx_taps[:, :, d])
        
tap_matrix = NetBuilder.syn_taps_to_nrn_taps(yx_taps, spacing=SYN_SPACE)
NetBuilder.make_taps_even(tap_matrix)

In [ ]:
# collect raw spikes
from pystorm.hal import HAL, RunControl
HAL = HAL()

net_builder = NetBuilder(HAL)

net = net_builder.create_single_pool_net(Y, X, tap_matrix, biases=BIAS)
pool = net.get_pools()[0]
inp = net.get_inputs()[0]

run = RunControl(HAL, net)

# map the network
HAL.map(net)

times = np.array([0, 1]) * 1e9
zero_rates = np.zeros((len(times), pool.dimensions))                                              
input_vals = {inp: (times, zero_rates)}
_, spikes_and_bin_times = run.run_input_sweep(input_vals, get_raw_spikes=True, get_outputs=False)
spikes, spike_bin_times = spikes_and_bin_times



In [ ]:
#HAL_inst = nengo_bs_instance.hal
HAL_inst = HAL
HAL_graph_objs = HAL_inst.last_mapped_network.get_GraphObjects()
HAL_conns = HAL_inst.last_mapped_network.get_connections()
f = open('HAL_net_dump.txt', 'w')
for obj in HAL_graph_objs:
    f.write(str(obj) + '\n')
for conn in HAL_conns:
    f.write(str(conn) + '\n')
f.close()

In [ ]:
def spike_bins_to_rates(spikes, orig_bin_times, desired_bin_times):
    rates = np.zeros((len(desired_bin_times) - 1, spikes.shape[1]))
    for t_idx, time in enumerate(desired_bin_times[:-1]):
        # XXX this is kind of overkill, should be able to infer indices
        valid_start_time = time 
        valid_end_time   = desired_bin_times[t_idx + 1]
        start_bin_idx = np.searchsorted(orig_bin_times, valid_start_time)
        end_bin_idx = np.searchsorted(orig_bin_times, valid_end_time)
        summed_spikes = np.sum(spikes[start_bin_idx:end_bin_idx], axis=0)
        rates[t_idx] = summed_spikes / ((valid_end_time - valid_start_time) / 1e9)
    return rates

BINS_PER_S = 20
num_bins = int(RUN_TIME * BINS_PER_S)
desired_bins = np.linspace(0, RUN_TIME, num_bins+1) * 1e9

spike_rates = spike_bins_to_rates(spikes[pool], spike_bin_times, desired_bins)

print(desired_bins.shape)
print(spike_rates.shape)
print(np.sum(spike_rates))

plt.figure()
plt.plot(desired_bins[:-1], spike_rates)
plt.title('binned spike rates')

plt.figure()
plt.plot(spike_bin_times, np.sum(spikes[pool], axis=1))
plt.title('all spikes summed, no additional binning')



In [ ]:
# now try collecting identity trick spikes

identity_dec = np.eye(N)
trick_net = net_builder.create_single_pool_net(Y, X, tap_matrix, biases=BIAS, decoders=identity_dec)
trick_inp = trick_net.get_inputs()[0]
trick_outp = trick_net.get_outputs()[0]

run = RunControl(HAL, trick_net)

HAL.map(trick_net)

times = np.array([0, 1]) * 1e9
zero_rates = np.zeros((len(times), pool.dimensions))                                              
input_vals = {trick_inp: (times, zero_rates)}
trick_spikes_and_bin_times, _ = run.run_input_sweep(input_vals, get_raw_spikes=False, get_outputs=True)
trick_spikes, trick_spike_bin_times = trick_spikes_and_bin_times

In [ ]:
from pystorm.hal import data_utils

def lpf_spike_bins_to_rates(spikes, orig_bin_times, tau):
    t = np.linspace(0, orig_bin_times[-1] - orig_bin_times[0], len(orig_bin_times))
    filt = (1 / tau) * np.exp(-t / 1e9 / tau)
    filt /= np.sum(filt)
    filt *= len(orig_bin_times)
    lpfd = np.zeros_like(spikes)
    for n in range(spikes.shape[1]):
        lpfd[:, n] = np.convolve(spikes[:, n], filt)[:spikes.shape[0]]
    return lpfd

trick_spike_rates = spike_bins_to_rates(trick_spikes[trick_outp], trick_spike_bin_times, desired_bins)
lpf_trick_spike_rates = lpf_spike_bins_to_rates(trick_spikes[trick_outp], trick_spike_bin_times, .05)
other_lpf_trick_spike_rates = data_utils.lpf(trick_spikes[trick_outp], .05, .001)

plt.figure(figsize=(14,14))

step_bins = np.zeros((2*desired_bins[:-1].shape[0],))
step_bins[::2] = desired_bins[:-1]
step_bins[1::2] = desired_bins[1:]
step_rates = np.zeros((step_bins.shape[0], trick_spike_rates.shape[1]))
step_rates[::2] = trick_spike_rates
step_rates[1::2] = trick_spike_rates

#plt.plot(desired_bins[:-1], spike_rates, 'r')
plt.plot(step_bins , step_rates, 'b')
plt.plot(trick_spike_bin_times, lpf_trick_spike_rates, 'g')
#plt.plot(trick_spike_bin_times, other_lpf_trick_spike_rates, 'r')

#plt.figure(figsize=(14,3))
#n = 12
#plt.plot(step_bins , step_rates[:, n], 'b')
#plt.plot(trick_spike_bin_times, lpf_trick_spike_rates[:, n], 'g')
#
##plt.plot(trick_spike_bin_times, other_lpf_trick_spike_rates, 'r')
#plt.xlim([.3e9, .5e9])
#mean = np.mean(lpf_trick_spike_rates[:, n])
#plt.ylim([.9 * mean, 1.1 * mean])
#plt.title('binned spike rates')
#
#ax1 = plt.gca()
#ax2 = plt.twinx()
#ax2.plot(trick_spike_bin_times, trick_spikes[trick_outp][:, n])

plt.figure(figsize=(10,10))
plt.plot(spike_bin_times, np.sum(spikes[pool], axis=1), 'r')
plt.plot(trick_spike_bin_times, np.sum(trick_spikes[trick_outp], axis=1), 'b')
plt.title('all spikes summed, no additional binning beyond FPGA binning')

def sum_and_make_plot(spikes, spike_bin_times, desired_bins, col):
    sum_spikes = np.sum(spikes, axis=1)
    sum_spikes = sum_spikes.reshape(len(sum_spikes), 1)
    sum_rates = spike_bins_to_rates(sum_spikes, spike_bin_times, desired_bins)
    plt.plot(desired_bins[:-1], sum_rates, col)
    
plt.figure()
sum_and_make_plot(spikes[pool], spike_bin_times, desired_bins, 'r')
sum_and_make_plot(trick_spikes[trick_outp], trick_spike_bin_times, desired_bins, 'b')
plt.title('all spikes summed and binned')

In [ ]:
# now do both at the same time

identity_dec = np.eye(N) * 1
trick_net = net_builder.create_single_pool_net(Y, X, tap_matrix, biases=BIAS, decoders=identity_dec)
pool = trick_net.get_pools()[0]
trick_inp = trick_net.get_inputs()[0]
trick_outp = trick_net.get_outputs()[0]

run = RunControl(HAL, trick_net)

HAL.map(trick_net)

times = np.array([0, 1]) * 1e9
zero_rates = np.zeros((len(times), pool.dimensions))                                              
input_vals = {trick_inp: (times, zero_rates)}
trick_spikes_and_bin_times, spikes_and_bin_times = run.run_input_sweep(input_vals, get_raw_spikes=True, get_outputs=True)
spikes, spike_bin_times = spikes_and_bin_times
trick_spikes, trick_spike_bin_times = trick_spikes_and_bin_times

In [ ]:
trick_spike_rates = spike_bins_to_rates(trick_spikes[trick_outp], trick_spike_bin_times, desired_bins)
spike_rates = spike_bins_to_rates(spikes[pool], spike_bin_times, desired_bins)

plt.figure()
plt.plot(desired_bins[:-1], spike_rates, 'r')
plt.plot(desired_bins[:-1], trick_spike_rates, 'b')
plt.title('binned spike rates')

plt.figure()
plt.plot(spike_bin_times, np.sum(spikes[pool], axis=1), 'r')
plt.plot(trick_spike_bin_times, np.sum(trick_spikes[trick_outp], axis=1), 'b')
plt.title('all spikes summed, no additional binning')

def sum_and_make_plot(spikes, spike_bin_times, desired_bins, col):
    sum_spikes = np.sum(spikes, axis=1)
    sum_spikes = sum_spikes.reshape(len(sum_spikes), 1)
    sum_rates = spike_bins_to_rates(sum_spikes, spike_bin_times, desired_bins)
    plt.plot(desired_bins[:-1], sum_rates, col)
    
plt.figure()
sum_and_make_plot(spikes[pool], spike_bin_times, desired_bins, 'r')
sum_and_make_plot(trick_spikes[trick_outp], trick_spike_bin_times, desired_bins, 'b')
plt.title('all spikes summed and binned')

In [ ]:
# now do a decode. See how things change with raw vs trick spikes

FMAX = 1000
HOLD_TIME = 1
SKIP_TIME = .5

from pystorm.hal import HAL, RunControl
HAL = HAL()

net_builder = NetBuilder(HAL)

net = net_builder.create_single_pool_net(Y, X, tap_matrix, biases=BIAS)
pool = net.get_pools()[0]
inp = net.get_inputs()[0]

run = RunControl(HAL, net)

# map the network
HAL.map(net)

train_vals = np.linspace(-1, 1, 40)
times = np.arange(len(input_vals)) * HOLD_TIME * 1e9
input_vals = {inp: (times, train_vals)
_, spikes_and_bin_times = run.run_input_sweep(input_vals, get_raw_spikes=True, get_outputs=False)
spikes, spike_bin_times = spikes_and_bin_times

Conclusions?

- raw spikes are pretty noisy, but if you're only collecting them (no simultaneous), the mean seems correct
- identity at the same time as raw spikes makes the raw spikes junk
    - gets better if it's more like .1 * identity
- there is correlated noise in the spikes
    - correlation is more obvious with fewer neurons, suggesting that it is synapse-linked
    - makes sense w.r.t to prior observation that jitter gets much worse with synapse connected
    - suggestion: broader diffusor, especially for 1D